## Aquí está el planteamiento general del script:

# Paso 1: Carga de Datos
     1.1. Descomprimir el archivo navs.zip para obtener los datos de cotizaciones de los fondos.

     1.2. Cargar los datos del archivo maestro.csv que contiene la información sobre los fondos.

# Paso 2: Preprocesamiento de Datos
     2.1. Mapear las categorías asset_type usando el diccionario asset_class.

     2.2. Crear funciones auxiliares para convertir las fechas, manejar las comisiones, y mapear las categorías family, nested_family, y subcategory.

# Paso 3: Implementación del Algoritmo K-NN
     3.1. Seleccionar las métricas a considerar para la búsqueda de fondos sustitutivos. Por ejemplo, se pueden utilizar métricas como el tipo de activo (asset_type), la clase del fondo (class_code), la comisión de gestión (management_fee), la zona geográfica (geo_area), etc.

     3.2. Normalizar las métricas para asegurar que todas tengan el mismo peso en el cálculo de la distancia.

     3.3. Implementar la función de distancia euclidiana para calcular la similitud entre fondos.

     3.4. Implementar la función K-NN que tome un ISIN como entrada y devuelva los N fondos más cercanos.

# Paso 4: Ejecución del Programa
     4.1. Cargamos Navs.pickle

     4.2. Cogemos todos los fondos de navs y selcelciona uno aleatoriamente.

     4.3.Utilizar la función K-NN para obtener los N fondos más cercanos.



In [4]:
import pandas as pd
import numpy as np
import zipfile

## Paso 1

In [5]:
# Descomprimir el archivo navs.zip
with zipfile.ZipFile('navs.zip', 'r') as zip_ref:
    zip_ref.extractall('navs')

In [6]:
maestro_df = pd.read_csv('maestro.csv')

In [4]:
maestro_df

,Unnamed: 0,isin,allfunds_id,asset,asset_type,class_code,clean_share,currency,geo_area,geo_zone,inception_at,income,management_fee,manager_id,manager_name,name,ongoing_charges
0,20,IE00B70DMB28,18152,7.0,ALS,A,False,EUR,3.0,JP,2017-10-11,False,2.00,3,NOMURA INVESTMENT SOLUTIONS PLC,"NOMURA ALPHA JPN LONG SHORT ""A"" (EURH) A",2.42
1,21,LU0331286657,206399,4.0,RVG,C2,False,EUR,3.0,ASP,2003-04-21,False,1.50,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""C2"" (EUR)",3.16
2,22,LU0252964605,27584,4.0,RVG,D2,True,EUR,3.0,ASP,2006-05-19,False,0.75,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""D2"" (EUR)",1.17
3,23,LU1495983089,12993,4.0,RVG,I2,False,USD,3.0,ASP,2016-09-28,False,0.75,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""I2"" (USD)",0.89
4,24,LU0171290587,8123,4.0,RVG,E2,False,EUR,3.0,ASP,2003-04-25,False,1.50,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""E2"" (EUR)",2.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24898,24918,LU1978535810,144810,4.0,RVG,IC,False,USD,1.0,US,2019-04-30,False,0.20,3545,DWS INVESTMENTS SA (REGISTER),"DWS INVEST ESG QI US ""IC"" (USD)",0.24
24899,24919,LU0502882342,249846,4.0,RVG,B,False,USD,1.0,US,2013-07-02,False,1.25,3638,WAYSTONE (LYRICAL VALUE FUNDS),"U.S. VALUE EQ STR ""B"" (USD)",1.38
24900,24920,LU1358059555,2913,7.0,NEU,B,True,EUR,2.0,EU,2016-06-01,False,1.50,3657,WAYSTONE DMS UCITS VELOX FUND,"VELOX FUND ""B"" (EURHDG)",2.01
24901,24921,LU1358059639,2914,7.0,NEU,B,True,USD,2.0,EU,2016-06-01,False,1.50,3657,WAYSTONE DMS UCITS VELOX FUND,"VELOX FUND ""B"" (USD)",2.01


## Paso 2

In [7]:
asset_class = {
        'ABR': 'alternative', 'ACF': 'alternative', 'ACU': 'alternative', 'AED': 'alternative', 'AGM': 'alternative', 'ALS': 'alternative',
        'ALT': 'alternative', 'AMF': 'alternative', 'AMS': 'alternative', 'ARV': 'alternative', 'BAL': 'mixed', 'BIN': 'equity', 'BIO': 'equity',
        'COM': 'other', 'CON': 'fixed_income', 'COR': 'fixed_income', 'COS': 'equity', 'DCO': 'equity', 'DEF': 'mixed', 'DRS': 'alternative',
        'DYN': 'mixed', 'EM': 'fixed_income', 'FIN': 'equity', 'FLE': 'mixed', 'GAR': 'fixed_income', 'GOL': 'equity', 'GOV': 'fixed_income',
        'HDG': 'alternative', 'HEA': 'equity', 'HY': 'fixed_income', 'IDT': 'equity', 'INF': 'fixed_income', 'MX': 'equity', 'MMD': 'money',
        'MMK': 'money', 'NEU': 'alternative', 'NAR': 'equity', 'RFG': 'fixed_income', 'RFL': 'fixed_income', 'RFM': 'fixed_income', 'RFS': 'fixed_income',
        'RST': 'equity', 'RVA': 'equity', 'RVG': 'equity', 'SMD': 'equity', 'TEC': 'equity', 'TEL': 'equity', 'TMA': 'mixed', 'UTI': 'equity', 'OTH': 'mixed', 'OTS': 'mixed',
    }

family = {
        'ABR': 'retorno absoluto', 'ACF': 'renta variable ocde', 'ACU': 'renta variable ocde', 'AED': 'renta variable ocde', 'AGM': 'renta variable ocde',
        'ALS': 'renta variable ocde', 'ALT': 'renta variable ocde', 'AMF': 'renta variable ocde', 'AMS': 'renta variable ocde', 'ARV': 'renta variable ocde',
        'BAL': 'deuda corporativa grado de inversion', 'BIN': 'renta variable ocde', 'BIO': 'renta variable ocde', 'COM': 'commodities', 'CON': 'convertibles',
        'COR': 'deuda corporativa grado de inversion', 'COS': 'renta variable ocde', 'DCO': 'renta variable ocde', 'DEF': 'deuda publica ocde', 'DRS': 'inmobiliario',
        'DYN': 'renta variable emergentes', 'EM': 'bonos high yield', 'FIN': 'renta variable ocde', 'FLE': 'renta variable ocde', 'GAR': 'retorno absoluto',
        'GOL': 'renta variable ocde', 'GOV': 'deuda publica ocde', 'HDG': 'retorno absoluto', 'HEA': 'renta variable ocde', 'HY': 'bonos high yield', 'IDT': 'renta variable ocde',
        'INF': 'deuda corporativa grado de inversion', 'MMD': 'liquidez y renta fija a corto plazo', 'MMK': 'liquidez y renta fija a corto plazo', 'MX': 'renta variable emergentes',
        'NAR': 'renta variable ocde', 'NEU': 'retorno absoluto', 'RFG': 'deuda corporativa grado de inversion', 'RFL': 'deuda corporativa grado de inversion',
        'RFM': 'deuda corporativa grado de inversion', 'RFS': 'deuda corporativa grado de inversion', 'RST': 'renta variable ocde', 'RVA': 'ver zona geografica',
        'RVG': 'ver zona geografica', 'SMD': 'ver zona geografica', 'TEC': 'renta variable ocde', 'TEL': 'renta variable ocde', 'TMA': 'deuda publica ocde',
        'UTI': 'renta variable ocde', 'OTH': 'renta variable ocde', 'OTS': 'renta variable ocde',
    }

subcategory = {
        'ABR': 'alternativos_liquidos', 'ACF': 'alternativos_liquidos', 'ACU': 'alternativos_liquidos', 'AED': 'alternativos_liquidos', 'AGM': 'alternativos_liquidos',
        'ALS': 'alternativos_liquidos', 'ALT': 'alternativos_liquidos', 'AMF': 'alternativos_liquidos', 'AMS': 'alternativos_liquidos', 'ARV': 'alternativos_liquidos',
        'BAL': 'mixtos_equilibrado', 'BIN': 'renta_variable_sectorial', 'BIO': 'renta_variable_sectorial', 'COM': 'otros', 'CON': 'renta_fija_convertibles',
        'COR': 'renta_fija_largo_plazo', 'COS': 'renta_variable_sectorial', 'DCO': 'renta_variable_sectorial', 'DEF': 'mixtos_conservador', 'DRS': 'alternativos_inmobiliario',
        'DYN': 'mixtos_agresivos', 'EM': 'renta_fija_emergente', 'FIN': 'renta_variable_sectorial', 'FLE': 'mixto_flexible', 'GAR': 'renta_fija_largo_plazo',
        'GOL': 'renta_variable_sectorial', 'GOV': 'renta_fija_largo_plazo', 'HDG': 'alternativos_liquidos', 'HEA': 'renta_variable_sectorial', 'HY': 'renta_fija_high_yield',
        'IDT': 'renta_variable_sectorial', 'INF': 'renta_fija_largo_plazo', 'MMD': 'monetario', 'MMK': 'monetario', 'MX': 'ver_zona_geografica', 
        'NAR': 'renta_variable_sectorial', 'NEU': 'alternativos_liquidos', 'RFG': 'renta_fija_largo_plazo', 'RFL': 'renta_fija_largo_plazo', 
        'RFM': 'renta_fija_corto_y_medio_plazo', 'RFS': 'renta_fija_corto_y_medio_plazo', 'RST': 'renta_variable_sectorial', 'RVA': 'ver_zona_geografica', 
        'RVG': 'ver_zona_geografica', 'SMD': 'ver_zona_geografica', 'TEC': 'renta_variable_sectorial', 'TEL': 'renta_variable_sectorial', 'TMA': 'mixtos_equilibrado', 
        'UTI': 'renta_variable_sectorial', 'OTH': 'mixto_flexible', 'OTS': 'mixto_flexible',
    }
full_name = {
    'BR': 'BRASIL', 'CA': 'CANADA', 'CAR': 'CARIBE', 'CL': 'CHILE', 'LAM': 'LATINOAMERICA MISC', 'LAT': 'LATINOAMERICA', 'MX': 'MEXICO', 'PAM': 'PANAMERICA', 
    'US': 'USA', 'ASE': 'ASIAN', 'ASP': 'ASIA PASIFICO', 'ATA': 'AUSTRALASIA', 'AU': 'AUSTRALIA', 'AXJ': 'ASIA EX JAPON', 'B': 'BAHREIN', 'CHI': 'CHINA', 
     'EAU': 'EMIRATOS ARABES UNIDOS', 'EGY': 'EGIPTO', 'EIJ': 'FAR EAST', 'EXJ': 'FAST EAST EX-JAPON', 'GCC': 'GCC', 'GCH': 'GREATER CHINA', 'HK': 'HONG KONG', 
     'IL': 'ISRAEL', 'IN': 'INDIA', 'IND': 'INDONESIA', 'JO': 'JORDANIA', 'JP': 'JAPON', 'KR': 'KOREA', 'KWT': 'KUWAIT', 'LE': 'LIBANO', 'MDE': 'ORIENTE MEDIO',
    'MNA': 'MENA', 'MO': 'MARRUECOS', 'MSA': 'MALASIA', 'OM': 'OMAN', 'PH': 'FILIPINAS', 'SAU': 'ARABIA SAUDITA', 'SG': 'SINGAPUR', 
    'SYM': 'SINGAPUR Y MALASIA', 'TH': 'THAILANDIA', 'TU': 'TUNEZ', 'TW': 'TAIWAN', 'VNM': 'VIETNAM', 'AT': 'AUSTRIA', 'BA': 'PAISES BALTICOS', 'BE': 'BELGICA', 
    'BNL': 'BENELUX', 'CH': 'SUIZA', 'CZE': 'REPUBLICA CHECA', 'DE': 'ALEMANIA', 'DK': 'DINAMARCA', 'EE': 'EUROPA EMERGENTE',
    'EES': 'EUROPA EXSUIZA', 'EMI': 'EUROPA MISC', 'ES': 'ESPAÑA', 'EU': 'EUROPA', 'EUK': 'EUROPA EX-UK', 'EUR': 'ZONA EURO', 'FI': 'FINLANDIA', 'FR': 'FRANCIA', 
    'GR': 'GRECIA', 'HUN': 'HUNGRIA', 'IBE': 'IBERICA', 'IE': 'IRLANDA', 'ISL': 'ISLANDIA', 'IT': 'ITALIA', 'LIE': 'LIETCHTENSTEIN', 'LUX': 'LUXEMBURGO', 
    'LU': 'LUXEMBURGO', 'MLT': 'MALTA', 'NL': 'PAISES BAJOS', 'NO': 'NORUEGA', 'NOR': 'PAISES NORDICOS', 'POL': 'POLONIA', 'PT': 'PORTUGAL', 'RS': 'RUSIA', 
    'SE': 'SUECIA', 'SKA': 'ESCANDINAVIA', 'SVK': 'ESLOVAQUIA', 'TR': 'TURQUIA', 'UK': 'REINO UNIDO', 'GB': 'REINO UNIDO', 'AFR': 'AFRICA', 'BRC': 'BRIC',
    'GEA': 'GLOBAL EX AUSTRALIA', 'GEM': 'GLOBAL EMERGENTE', 'GEU': 'GLOBAL EX-US', 'GLB': 'GLOBAL', 'NZL': 'NUEVA ZELANDA', 'SAF': 'SUDAFRICA', 'EME': 'EMEA'
}

nested_family = {
        'BR': 'renta variable emergentes', 'CA': 'renta variable ocde', 'CAR': 'renta variable emergentes', 'CL': 'renta variable emergentes', 
        'LAM': 'renta variable emergentes', 'LAT': 'renta variable emergentes', 'MX': 'renta variable emergentes', 'PAM': 'renta variable emergentes', 
        'US': 'renta variable ocde', 'ASE': 'renta variable emergentes', 'ASP': 'renta variable emergentes', 'ATA': 'renta variable emergentes', 'AU': 'renta variable ocde', 
        'AXJ': 'renta variable emergentes', 'B': 'renta variable emergentes', 'CHI': 'renta variable emergentes', 'EAU': 'renta variable emergentes', 
        'EGY': 'renta variable emergentes', 'EIJ': 'renta variable emergentes', 'EXJ': 'renta variable emergentes', 'GCC': 'renta variable emergentes', 
        'GCH': 'renta variable emergentes', 'HK': 'renta variable emergentes', 'IL': 'renta variable emergentes', 'IN': 'renta variable emergentes',
        'IND': 'renta variable emergentes', 'JO': 'renta variable emergentes', 'JP': 'renta variable emergentes', 'KR': 'renta variable emergentes', 
        'KWT': 'renta variable emergentes', 'LE': 'renta variable emergentes', 'MDE': 'renta variable emergentes', 'MNA': 'renta variable emergentes', 
        'MO': 'renta variable emergentes', 'MSA': 'renta variable emergentes', 'OM': 'renta variable emergentes', 'PH': 'renta variable emergentes', 
        'SAU': 'renta variable emergentes', 'SG': 'renta variable emergentes', 'SYM': 'renta variable emergentes', 'TH': 'renta variable emergentes', 
        'TU': 'renta variable emergentes', 'TW': 'renta variable emergentes', 'VNM': 'renta variable emergentes', 'AT': 'renta variable ocde',
        'BA': 'renta variable emergentes', 'BE': 'renta variable ocde', 'BNL': 'renta variable ocde', 'CH': 'renta variable ocde', 'CZE': 'renta variable ocde', 
        'DE': 'renta variable ocde', 'DK': 'renta variable ocde', 'EE': 'renta variable emergentes', 'EES': 'renta variable ocde', 'EMI': 'renta variable ocde', 
        'ES': 'renta variable ocde', 'EU': 'renta variable ocde', 'EUK': 'renta variable ocde', 'EUR': 'renta variable ocde', 'FI': 'renta variable ocde', 
        'FR': 'renta variable ocde', 'GR': 'renta variable ocde', 'HUN': 'renta variable ocde', 'IBE': 'renta variable ocde', 'IE': 'renta variable ocde', 
        'ISL': 'renta variable ocde', 'IT': 'renta variable ocde', 'LIE': 'renta variable ocde', 'LUX': 'renta variable ocde', 'LU': 'renta variable ocde', 
        'MLT': 'renta variable ocde', 'NL': 'renta variable ocde', 'NO': 'renta variable ocde', 'NOR': 'renta variable ocde', 'POL': 'renta variable ocde',
        'PT': 'renta variable ocde', 'RS': 'renta variable emergentes', 'SE': 'renta variable ocde', 'SKA': 'renta variable ocde', 'SVK': 'renta variable ocde', 
        'TR': 'renta variable emergentes', 'UK': 'renta variable ocde', 'GB': 'renta variable ocde', 'AFR': 'renta variable emergentes', 'BRC': 'renta variable emergentes', 
        'GEA': 'renta variable ocde', 'GEM': 'renta variable emergentes', 'GEU': 'renta variable ocde', 'GLB': 'renta variable ocde', 'NZL': 'renta variable ocde', 
        'SAF': 'renta variable emergentes', 'EME': 'renta variable ocde'
    }


In [8]:
maestro_df['asset_class'] = maestro_df['asset_type'].map(asset_class)
maestro_df['family'] = maestro_df['asset_type'].map(family)
maestro_df['subcategory'] = maestro_df['asset_type'].map(subcategory)
maestro_df['nested_family'] = maestro_df['geo_zone'].map(nested_family)
maestro_df['geo_zone'] = maestro_df['geo_zone'].map(full_name)


In [7]:
maestro_df

,Unnamed: 0,isin,allfunds_id,asset,asset_type,class_code,clean_share,currency,geo_area,geo_zone,...,income,management_fee,manager_id,manager_name,name,ongoing_charges,asset_class,family,subcategory,nested_family
0,20,IE00B70DMB28,18152,7.0,ALS,A,False,EUR,3.0,JAPON,...,False,2.00,3,NOMURA INVESTMENT SOLUTIONS PLC,"NOMURA ALPHA JPN LONG SHORT ""A"" (EURH) A",2.42,alternative,renta variable ocde,alternativos_liquidos,renta variable emergentes
1,21,LU0331286657,206399,4.0,RVG,C2,False,EUR,3.0,ASIA PASIFICO,...,False,1.50,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""C2"" (EUR)",3.16,equity,ver zona geografica,ver_zona_geografica,renta variable emergentes
2,22,LU0252964605,27584,4.0,RVG,D2,True,EUR,3.0,ASIA PASIFICO,...,False,0.75,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""D2"" (EUR)",1.17,equity,ver zona geografica,ver_zona_geografica,renta variable emergentes
3,23,LU1495983089,12993,4.0,RVG,I2,False,USD,3.0,ASIA PASIFICO,...,False,0.75,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""I2"" (USD)",0.89,equity,ver zona geografica,ver_zona_geografica,renta variable emergentes
4,24,LU0171290587,8123,4.0,RVG,E2,False,EUR,3.0,ASIA PASIFICO,...,False,1.50,7,BLACKROCK GLOBAL FUNDS,"BGF PACIFIC EQUITY ""E2"" (EUR)",2.41,equity,ver zona geografica,ver_zona_geografica,renta variable emergentes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24898,24918,LU1978535810,144810,4.0,RVG,IC,False,USD,1.0,USA,...,False,0.20,3545,DWS INVESTMENTS SA (REGISTER),"DWS INVEST ESG QI US ""IC"" (USD)",0.24,equity,ver zona geografica,ver_zona_geografica,renta variable ocde
24899,24919,LU0502882342,249846,4.0,RVG,B,False,USD,1.0,USA,...,False,1.25,3638,WAYSTONE (LYRICAL VALUE FUNDS),"U.S. VALUE EQ STR ""B"" (USD)",1.38,equity,ver zona geografica,ver_zona_geografica,renta variable ocde
24900,24920,LU1358059555,2913,7.0,NEU,B,True,EUR,2.0,EUROPA,...,False,1.50,3657,WAYSTONE DMS UCITS VELOX FUND,"VELOX FUND ""B"" (EURHDG)",2.01,alternative,retorno absoluto,alternativos_liquidos,renta variable ocde
24901,24921,LU1358059639,2914,7.0,NEU,B,True,USD,2.0,EUROPA,...,False,1.50,3657,WAYSTONE DMS UCITS VELOX FUND,"VELOX FUND ""B"" (USD)",2.01,alternative,retorno absoluto,alternativos_liquidos,renta variable ocde


In [9]:
def convertir_fecha(fecha):
    return pd.to_datetime(fecha, format='%Y-%m-%d')
maestro_df['inception_at'] = maestro_df['inception_at'].apply(convertir_fecha)

In [10]:
maestro_df['clean_share'] = maestro_df['clean_share'].astype(int) # 1 si tiene comisión, 0 si no

In [11]:
maestro_df['income'] = maestro_df['income'].astype(int) # 1 si reparte dividendos, 0 si no

In [45]:
maestro_df

RangeIndex(start=0, stop=24903, step=1)

In [12]:
asset_class = {str(k): v for k, v in asset_class.items()}
family = {str(k): v for k, v in family.items()}
subcategory = {str(k): v for k, v in subcategory.items()}

In [13]:
maestro_df['asset_class'] = maestro_df['asset_type'].map(asset_class).astype('category')
maestro_df['family'] = maestro_df['asset_type'].map(family).astype('category')
maestro_df['subcategory'] = maestro_df['asset_type'].map(subcategory).astype('category')

In [14]:
maestro_df.isna().sum()


Unnamed: 0          0
isin                0
allfunds_id         0
asset               5
asset_type         14
class_code         16
clean_share         0
currency            0
geo_area            5
geo_zone           14
inception_at        0
income              0
management_fee      4
manager_id          0
manager_name        0
name                0
ongoing_charges    30
asset_class        14
family             14
subcategory        14
nested_family      14
dtype: int64

## Eliminanos las filas con NA's

In [15]:
maestro_df = maestro_df.dropna()

In [16]:
maestro_df.isna().sum()

Unnamed: 0         0
isin               0
allfunds_id        0
asset              0
asset_type         0
class_code         0
clean_share        0
currency           0
geo_area           0
geo_zone           0
inception_at       0
income             0
management_fee     0
manager_id         0
manager_name       0
name               0
ongoing_charges    0
asset_class        0
family             0
subcategory        0
nested_family      0
dtype: int64

# 
Ya una vez tratados los datos provenientes del csv , hay que realizar el algoritmo de KNN, primero tocará escoger las metricas del analisis para buscar fondos sustitutivos y normalizarlas para que tengan el mismo peso, Establecer la tipología de la distancia que aplicaremos (distancia euclidiana) y un valor a k(k=5), y por último implementar la función K-NN que tome un ISIN como entrada y devuelva los N fondos más cercanos.  

## Paso 3

In [17]:
from sklearn.preprocessing import MinMaxScaler

metric_columns = ['isin', 'asset', 'clean_share', 'management_fee', 'income']
#Las métricas escogidas son el ISIN para buscar similares, el riesgo, si tieno o no comision, si reparte o no dividendos y % de comision por Gestión.
# Crear un nuevo DataFrame con las métricas seleccionadas
metrics_df = maestro_df[metric_columns]

# Extraer la columna 'isin' antes de la normalización
isin_column = metrics_df['isin']
metrics_df = metrics_df.drop('isin', axis=1)

# Normalizar utilizando Min-Max Scaling
scaler = MinMaxScaler()
normalized_metrics = pd.DataFrame(scaler.fit_transform(metrics_df), columns=metric_columns[1:])

# Restaurar la columna 'isin'
normalized_metrics['isin'] = isin_column

In [18]:
normalized_metrics

,asset,clean_share,management_fee,income,isin
0,1.000000,0.0,0.689655,0.0,IE00B70DMB28
1,0.571429,0.0,0.517241,0.0,LU0331286657
2,0.571429,1.0,0.258621,0.0,LU0252964605
3,0.571429,0.0,0.258621,0.0,LU1495983089
4,0.571429,0.0,0.517241,0.0,LU0171290587
...,...,...,...,...,...
24840,0.571429,0.0,0.068966,0.0,LU2240056791
24841,0.571429,0.0,0.431034,0.0,LU2240056445
24842,1.000000,1.0,0.517241,0.0,LU2240056015
24843,1.000000,1.0,0.517241,0.0,LU2240056106


In [19]:
#Implementar la función de distancia euclidiana para calcular la similitud entre fondos a través de numpys

def euclidean_distance(x, y):
    return np.sqrt(np.sum((x - y) ** 2))


In [20]:
def k_nearest_neighbors(isin, k=5):
    # Obtener las métricas normalizadas para el fondo dado
    fund_metrics = normalized_metrics.loc[normalized_metrics['isin'] == isin].select_dtypes(include=[np.number]).values.flatten()

    # Calcular la distancia euclidiana con todos los fondos
    distances = np.apply_along_axis(lambda x: euclidean_distance(fund_metrics, x), 1, normalized_metrics.select_dtypes(include=[np.number]).values)

    # Obtener los índices de los fondos más cercanos
    nearest_indices = np.argsort(distances)[:k]

    # Devolver los ISIN de los fondos más cercanos
    nearest_isins = maestro_df.loc[nearest_indices, 'isin'].tolist()

    return nearest_isins

# Ejemplo de uso
user_isin = 'LU0376447149'  # Reemplaza con el ISIN del usuario
nearest_funds = k_nearest_neighbors(user_isin, k=5)
print("Fondos sustitutivos encontrados:")
print(nearest_funds)


Fondos sustitutivos encontrados:
['IE00B61KDS97', 'LU0827876151', 'LU1809976951', 'LU0406496546', 'LU1957153940']


## Paso 4

In [21]:
import pickle
import pandas as pd


# Cargar los datos de navs.pickle
with open('navs.pickle', 'rb') as f:
    navs_data = pickle.load(f)

In [22]:
# Obtener una lista de claves (que son los posibles números)
available_numbers = list(navs_data.keys())

# Seleccionar aleatoriamente un número del conjunto disponible
selected_number = np.random.choice(available_numbers)

# Obtener el ISIN correspondiente al número seleccionado
user_isin = navs_data[selected_number]['isin'].iloc[0]
#Ejecutamos la función en base al ISIN generado aleatoriamente de navs.pickle
nearest_funds = k_nearest_neighbors(user_isin, k=5)

# Imprimir los resultados
print(f"ISIN aleatorio seleccionado: {user_isin}")
print("Fondos sustitutivos encontrados:")
print(nearest_funds)

ISIN aleatorio seleccionado: LU0740037022
Fondos sustitutivos encontrados:
['LU0826412388', 'LU1842103399', 'LU0996172259', 'LU1842103985', 'LU0945154671']
